## 함수호출 하는 챗봇 구성
• 임의의 함수를 정의하고 설정해서 응답 메시지 결과에 반영이 되도록 정의


In [41]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

### 나이 계산 함수
예시: "1992-05-01생의 만 나이는? <br>
함수명: calculate_age 입력: birthdate(문자열, YYYY-MM-DD)

In [55]:
# 함수 정의

from datetime import datetime

def calculate_age(birthdate):
    today = datetime.today()
    birth = datetime.strptime(birthdate, "%Y-%m-%d")
    age = today.year - birth.year - ((today.month, today.day) < (birth.month, birth.day))
    return age

In [56]:
# 함수 호출 요청
tools = [{
    "type": "function",
    "name": "calculate_age",
    "description": "calculate the age based on the provided birthdate",
    "parameters": {
        "type": "object",
        "properties": {
            "birthdate": {"type": "string"}
        },
        "required": ["birthdate"],
        "additionalProperties": False
    },
    "strict": True
}]


In [57]:
# 응답
input_messages = [
    {'role': 'user', 'content': '1990-05-15 생의 만 나이는 몇 살인가요?'}
]

response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)
print(response.output[0])

ResponseFunctionToolCall(arguments='{"birthdate":"1990-05-15"}', call_id='call_Vm7YpwYfy9YrUB0oHE8CXopt', name='calculate_age', type='function_call', id='fc_685bf5738908819bacc294f69bdd56280826b7fdd4eeeb6a', status='completed')


In [58]:
import json

# call에 대한 응답 데이터
tool_call = response.output[0]

args = json.loads(tool_call.arguments)

result = calculate_age(args["birthdate"])
print(result)


35


In [59]:
input_messages.append(tool_call)

input_messages.append(
    {
        'type': 'function_call_output',
        'call_id': tool_call.call_id,
        'output': str(result)
    }
)

# 최종 응답
response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

1990년 5월 15일생의 만 나이는 35세입니다.


### 환율 변환 함수
예시: "100달러를 원화로 바꿔줘 (환율 1330원 적용)"
함수명: convert_currency 입력: amount(숫자)

In [60]:
def convert_currency(amount, from_currency, to_currency, rate):
    converted = amount * rate
    return {
        "amount": amount,
        "from_currency": from_currency,
        "to_currency": to_currency,
        "rate": rate,
        "converted_amount": converted
    }

In [65]:
tools = [{
    "type": "function",
    "name": "convert_currency",
    "description": "calculate the converted amount based on the provided parameters",
    "parameters": {
        "type": "object",
        "properties": {
            "amount" : {"type":"number"},
            "from_currency" : {"type":"string"},
            "to_currency" : {"type":"string"},
            "rate" : {"type":"number"}
        },
        "required": ["amount", "from_currency", "to_currency", "rate"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [
    {'role': 'user', 'content': '100달러를 원화로 바꿔줘 (환율 1330원 적용)'}
]

response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(response.output[0])

ResponseFunctionToolCall(arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW","rate":1330}', call_id='call_IegfrtJ6dflnZdzsw42wOitf', name='convert_currency', type='function_call', id='fc_685bf684dfb8819a9ef32a00ca953be00aaba6d39dbb0e26', status='completed')


In [66]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = convert_currency(args["amount"], args["from_currency"], args["to_currency"], args["rate"])
# print(result)

input_messages.append(tool_call)

input_messages.append(
    {
        'type': 'function_call_output',
        'call_id': tool_call.call_id,
        'output': str(result)
    }
)

# 최종 응답
response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

100달러를 1330원의 환율로 원화로 바꾸면 133,000원이 됩니다.


### BMI(체질량지수) 계산 함수
예시: "키 170cm, 몸무게 65kg의 BMI는?“
함수명: calculate_bmi 입력: height(숫자, cm), weight(숫자, kg)

In [67]:
def calculate_bmi  (weight, height):
    if height <= 0:
        return "Height must be greater than zero."
    bmi = weight / (height ** 2)
    return round(bmi, 2)

In [68]:
tools = [{
    "type": "function",
    "name": "calculate_bmi",
    "description": "calculate the Body Mass Index (BMI) based on weight and height",
    "parameters": {
        "type": "object",
        "properties": {
            "weight": {"type": "number"},
            "height": {"type": "number"}
        },
        "required": ["weight", "height"],
        "additionalProperties": False
    },
    "strict": True
}]

In [69]:
input_messages = [
    {'role': 'user', 'content': '키 170cm, 몸무게 65kg의 BMI는?'}
]

response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(response.output[0])

ResponseFunctionToolCall(arguments='{"weight":65,"height":170}', call_id='call_oA4e47mmpN47iPSilGK82SsE', name='calculate_bmi', type='function_call', id='fc_685bf72bb6a0819abb8f6b031d930b2f02e32f6ec76e3ccd', status='completed')


In [71]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = calculate_bmi(args["weight"], args["height"])
# print(result)

input_messages.append(tool_call)

input_messages.append(
    {
        'type': 'function_call_output',
        'call_id': tool_call.call_id,
        'output': str(result)
    }
)

# 최종 응답
response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

키 170cm, 몸무게 65kg의 BMI는 22.5입니다.

BMI 계산 공식: (몸무게(kg)) ÷ (키(m) × 키(m))
계산 과정: 65 ÷ (1.7 × 1.7) = 22.5

이 수치는 정상 체중 범위에 해당합니다.
